In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

In [29]:
#cargando. excels
#xls = pd.ExcelFile('Copert5.xlsx')
xls = pd.ExcelFile('1.A.3.b.i-iv Road Transport Appendix 4 Emission Factors 2021.xlsx')
df = pd.read_excel(xls,'HOT_EMISSIONS_PARAMETERS', index_col=None)

In [30]:
#Filtrando usando REGEX a las categorías vehiculares a utilizar
df = df.loc[
      ((df['Fuel'] == 'Petrol') |  (df['Fuel'] == 'Diesel')) &
      df.Segment.str.contains('^Medium') | #VLP, TAX
      df.Segment.str.contains("^N1-II$")  | #VLC
      df.Segment.str.contains('^Art*|^Rigid 28 *|^Rigid >32*') | #CAM PES RIGID>28 t and articulated
      df.Segment.str.contains('^Rigid 14 *|^Rigid 20 *|^Rigid 26 *') |#CAM MED RIGID 14-28 t
      df.Segment.str.contains('^Rigid <=7,5 *|^Rigid 7,5 -*|^Rigid 12 *') |#CAM LIV
      df.Segment.str.contains('^Urban Buses Standard 15 *') | #Bus RIG
      df.Segment.str.contains('^Urban Buses Articulated >18 t') | #BUS ART
      df.Segment.str.contains('^Coaches Standard <=18 t') | #BUS INT PROV
      df.Segment.str.contains("^Motorcycles 2-stroke >50 *") #MOT
]


In [31]:
#Cambiando a la clasificacion a utilizar
df.loc[df['Segment'] == 'Medium', 'Segment'] = 'VLP'
df.loc[df['Segment'] == 'N1-II', 'Segment'] = 'VLC'
df.loc[df.Segment.str.contains('^Rigid <=7,5 *|^Rigid 7,5 -*|^Rigid 12 *'), 'Segment'] = 'CAM LIV'
df.loc[df.Segment.str.contains('^Rigid 14 *|^Rigid 20 *|^Rigid 26 *'), 'Segment'] = 'CAM MED'
df.loc[df.Segment.str.contains('^Art*|^Rigid 28 *|^Rigid >32*'), 'Segment'] = 'CAM PES'
df.loc[df.Segment.str.contains('^Urban Buses Standard 15 *'), 'Segment'] = 'BUS RIG'
df.loc[df.Segment.str.contains('^Urban Buses Articulated >18 t'), 'Segment'] = 'BUS ART'
df.loc[df.Segment.str.contains('^Coaches Standard <=18 t'), 'Segment'] = 'BUS INT PROV'
df.loc[df.Segment.str.contains("^Motorcycles 2-stroke >50 *"), 'Segment'] = 'MOT'
#df.to_excel('C5_reducido.xlsx',index = False)
#df.loc[df.Segment.str.contains('^CAM MED*')] 

In [32]:
df = df.loc[ ((df['Load'].isnull())) | #todas las pendientes y las que no tienen pendiente
       ((df['Load'] == 0.5))] #carga 50% y las que su carga no está definida
pd.unique(df["Load"])
df = df.loc[(df["Mode"] == "Highway") | (df["Mode"].isnull()) | (df["Mode"] == "Urban Peak") | (df["Mode"] == "Hot")]
df["Mode"].unique()

array([nan, 'Urban Peak', 'Highway', 'Hot'], dtype=object)

In [33]:
#Cambiando clasificaciones
df.loc[df['Euro Standard'] == 'PRE ECE', 'Euro Standard'] = 'No cat'
df.loc[df['Euro Standard'] == 'Euro 1', 'Euro Standard'] = 'E1'
df.loc[df['Euro Standard'] == 'Euro 2', 'Euro Standard'] = 'E2'
df.loc[df['Euro Standard'] == 'Euro 3', 'Euro Standard'] = 'E3'
df.loc[df['Euro Standard'] == 'Euro 4', 'Euro Standard'] = 'E4'
df.loc[df['Euro Standard'] == 'Euro 5', 'Euro Standard'] = 'E5'
df.loc[df['Euro Standard'] == 'Euro 6 a/b/c', 'Euro Standard'] = 'E6'
df.loc[df['Euro Standard'] == 'Euro 6 d', 'Euro Standard'] = 'E6'
df.loc[df['Euro Standard'] == 'Conventional', 'Euro Standard'] = 'No cat'
df.loc[df['Euro Standard'] == 'Euro I', 'Euro Standard'] = 'E1'
df.loc[df['Euro Standard'] == 'Euro II', 'Euro Standard'] = 'E2'
df.loc[df['Euro Standard'] == 'Euro III', 'Euro Standard'] = 'E3'
df.loc[df['Euro Standard'] == 'Euro IV', 'Euro Standard'] = 'E4'
df.loc[df['Euro Standard'] == 'Euro V', 'Euro Standard'] = 'E5'
df.loc[df['Euro Standard'] == 'Euro VI A/B/C', 'Euro Standard'] = 'E6' #al parecer no hay diferencia en los parámetros estudiados
df.loc[df['Euro Standard'] == 'Euro VI D/E', 'Euro Standard'] = 'E6'
df.loc[df['Fuel'] == 'Petrol', 'Fuel'] = 'Gasolina'
#df.to_excel('C5_reducido_prueba.xlsx',index = False)

In [34]:
df = df.loc[
    (df["Euro Standard"] == "No cat") |
    (df["Euro Standard"] == "E1") |
    (df["Euro Standard"] == "E2") |
    (df["Euro Standard"] == "E3") |
    (df["Euro Standard"] == "E4") |
    (df["Euro Standard"] == "E5") |
    (df["Euro Standard"] == "E6") |
    (df["Euro Standard"] == "E6")
]

In [35]:
df = df.loc[
            #Todo lo en blanco
            (df['Technology'].isnull()) |
            #Passenger Cars
            ((df['Category'] == 'Passenger Cars') & 
             (df['Technology'] == 'PFI') | (df['Technology'] == 'DPF')) |
            #Light Commercial Vehicles
            ((df['Category'] == 'Light Commercial Vehicles') & 
             (df['Technology'] == 'PFI') | (df['Technology'] == 'DPF')) |
             #Buses
            ((df['Category'] == 'Buses') & 
             (df['Technology'] == 'EGR') | (df['Technology'] == 'DPF+SCR')) |
            #Heavy Duty Trucks
            ((df['Category'] == 'Heavy Duty Trucks') & 
             (df['Technology'] == 'EGR') | (df['Technology'] == 'DPF+SCR'))
            ]

In [36]:
df2 = pd.read_excel("input/Velocidades.xlsx")
df2

,Region,Ambito,Modo,Motorizacion,Norma,Gas,Unidad,2014,2015,2016,...,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060
0,R01,Urbano,BUS ART,Diesel,E1,Velocidad,[km/hr],25,25,25,...,25,25,25,25,25,25,25,25,25,25
1,R01,Urbano,BUS ART,Diesel,E2,Velocidad,[km/hr],25,25,25,...,25,25,25,25,25,25,25,25,25,25
2,R01,Urbano,BUS ART,Diesel,E3,Velocidad,[km/hr],25,25,25,...,25,25,25,25,25,25,25,25,25,25
3,R01,Urbano,BUS ART,Diesel,E4,Velocidad,[km/hr],25,25,25,...,25,25,25,25,25,25,25,25,25,25
4,R01,Urbano,BUS ART,Diesel,E5,Velocidad,[km/hr],25,25,25,...,25,25,25,25,25,25,25,25,25,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2092,R09,Urbano,VLC,Gasolina,E4,Velocidad,[km/hr],25,25,25,...,25,25,25,25,25,25,25,25,25,25
2093,R09,Urbano,VLC,Gasolina,E5,Velocidad,[km/hr],25,25,25,...,25,25,25,25,25,25,25,25,25,25
2094,R09,Urbano,VLC,Gasolina,E6,Velocidad,[km/hr],25,25,25,...,25,25,25,25,25,25,25,25,25,25
2095,R09,Urbano,VLC,Gasolina,No cat,Velocidad,[km/hr],25,25,25,...,25,25,25,25,25,25,25,25,25,25


In [37]:
bdf = df2.merge(df, left_on=["Modo", "Motorizacion", "Norma"], right_on=["Segment","Fuel","Euro Standard"], how="left")
alpha = bdf[["Alpha"]].astype("float")
beta = bdf[["Beta"]].astype("float")
gamma = bdf[["Gamma"]].astype("float")
delta = bdf[["Delta"]].astype("float")
epsilon = bdf[["Epsilon"]].astype("float")
zeta = bdf[["Zita"]].astype("float")
hta = bdf[["Hta"]].astype("float")
redFactor = bdf[["Reduction Factor [%]"]].astype("float")
bdf.columns
vel = bdf.iloc[:,7:len(df2.columns)]
df2.loc[(df2["Modo"]=="VLP") & (df2["Norma"]=="E1") & (df2["Region"]=="R13")]
bdf.loc[(bdf["Modo"]=="VLP") & (bdf["Norma"]=="E1") & (bdf["Region"]=="R13") & (bdf["Pollutant"]=="CO")]

,Region,Ambito,Modo,Motorizacion,Norma,Gas,Unidad,2014,2015,2016,...,Epsilon,Zita,Hta,Reduction Factor [%],Bio Reduction Factor [%],50,EF [g/km] or ECF [MJ/km] or #/km or #/kWh or g/kWh,Unnamed: 22,Unnamed: 23,Unnamed: 24
67484,R13,Interurbano,VLP,Diesel,E1,Velocidad,[km/hr],100,100,100,...,-1.515809e-16,1.013966e-15,1.537124,0.0,0.0,50.0,0.328287,NaN,NaN,NaN
67569,R13,Interurbano,VLP,Gasolina,E1,Velocidad,[km/hr],100,100,100,...,-7.381687e-04,1.003078e-01,0.779481,0.0,0.0,50.0,1.533630,NaN,NaN,NaN
75256,R13,Urbano,VLP,Diesel,E1,Velocidad,[km/hr],25,25,25,...,-1.515809e-16,1.013966e-15,1.537124,0.0,0.0,50.0,0.328287,NaN,NaN,NaN
75341,R13,Urbano,VLP,Gasolina,E1,Velocidad,[km/hr],25,25,25,...,-7.381687e-04,1.003078e-01,0.779481,0.0,0.0,50.0,1.533630,NaN,NaN,NaN


In [114]:
vel2 = np.where(vel.gt(bdf["Max Speed [km/h]"], axis=0),bdf[["Max Speed [km/h]"]],vel)
vel2 = pd.DataFrame(vel2)
vel2.columns = vel.columns
discriminante = (epsilon.values*vel2**2+zeta.values*vel2+hta.values)
EFs = np.where(discriminante == 0, 0, (1 - redFactor.values)*((alpha.values * vel2**2 + beta.values * vel2 
+ gamma.values + (delta.values/vel2)) / (discriminante)))
EFs = pd.DataFrame(EFs)
EFs.columns = vel.columns
EFs = pd.concat([bdf.iloc[:,[0,1,2,3,4,-20]],EFs],axis=1)
EFs = EFs.groupby(["Region","Ambito","Modo","Motorizacion",'Norma',"Pollutant"],as_index=False).mean()
EFs.loc[(EFs["Modo"]=="BUS RIG") & (EFs["Region"]=="R13") & (EFs["Pollutant"]=="PM Exhaust")]

,Region,Ambito,Modo,Motorizacion,Norma,Pollutant,2014,2015,2016,2017,...,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060
13375,R13,Urbano,BUS RIG,Diesel,E1,PM Exhaust,0.445616,0.445616,0.445616,0.445616,...,0.445616,0.445616,0.445616,0.445616,0.445616,0.445616,0.445616,0.445616,0.445616,0.445616
13384,R13,Urbano,BUS RIG,Diesel,E2,PM Exhaust,0.223260,0.223260,0.223260,0.223260,...,0.223260,0.223260,0.223260,0.223260,0.223260,0.223260,0.223260,0.223260,0.223260,0.223260
13393,R13,Urbano,BUS RIG,Diesel,E3,PM Exhaust,0.212056,0.212056,0.212056,0.212056,...,0.212056,0.212056,0.212056,0.212056,0.212056,0.212056,0.212056,0.212056,0.212056,0.212056
13402,R13,Urbano,BUS RIG,Diesel,E4,PM Exhaust,0.053178,0.053178,0.053178,0.053178,...,0.053178,0.053178,0.053178,0.053178,0.053178,0.053178,0.053178,0.053178,0.053178,0.053178
13429,R13,Urbano,BUS RIG,Diesel,No cat,PM Exhaust,0.824530,0.824530,0.824530,0.824530,...,0.824530,0.824530,0.824530,0.824530,0.824530,0.824530,0.824530,0.824530,0.824530,0.824530


In [67]:
#Calculo CO2 y SOx
gasDensidad = 0.730 #Ton/m3
gasPcalor = 11200 #Kcal/kg 
dieselPcalor = 10900 #Kcal/kg 
dieselDensidad = 0.840 #ton/m3
tJouleAtCal = 2.39*10**(-1) #de joule a cal 
kCalLt = (gasDensidad*1000*gasPcalor*0.001)
kCalLt = (dieselDensidad*1000*dieselPcalor*0.001)
######## Fin Datos ##########
#Calculando CO2, SOx, km/lt
ECs_gas=EFs[(EFs["Pollutant"]=="EC") & (EFs["Motorizacion"] == "Gasolina")].iloc[:,6:-1]
ECs_diesel = EFs[(EFs["Pollutant"]=="EC") & (EFs["Motorizacion"] == "Diesel")].iloc[:,6:-1]
FCgas = (1000*ECs_gas*tJouleAtCal)/(gasPcalor/1000) #gr/km
FCdies = (1000*ECs_diesel*tJouleAtCal)/(dieselPcalor/1000) #gr/km
CO2_gas = 44.0011*(FCgas/(12.011+1.008*1.8))
CO2_dies = 44.0011*(FCdies/(12.011+1.008*2))
SOx_gas = 2*5*(10**-5)*FCgas
SOx_dies = 2*5*(10**-5)*FCdies
kmltGas = (FCgas/(gasDensidad*10**3))**-1
kmltDies = (FCdies/(dieselDensidad*10**3))**-1
namesG = EFs[(EFs["Pollutant"]=="EC") & (EFs["Motorizacion"] == "Gasolina")].iloc[:,0:6]
namesD = EFs[(EFs["Pollutant"]=="EC") & (EFs["Motorizacion"] == "Diesel")].iloc[:,0:6]
CO2 = pd.concat([pd.concat([namesG,CO2_gas],axis=1), pd.concat([namesD,CO2_dies],axis=1)],axis=0)
CO2["Pollutant"] = "CO2"
SOx = pd.concat([pd.concat([namesG,SOx_gas],axis=1), pd.concat([namesD,SOx_dies],axis=1)],axis=0)
SOx["Pollutant"] = "SOx"
kmLt = pd.concat([pd.concat([namesG,kmltGas],axis=1), pd.concat([namesD,kmltDies],axis=1)],axis=0)
kmLt["Pollutant"] = "[km/lt]"

In [97]:
#Fixing g/kWh PM to gr/km
consumo = EFs.loc[
    ((EFs.Modo.str.contains("CAM*")) | (EFs.Modo.str.contains("BUS*"))) & (EFs["Pollutant"] == "EC") & 
    ((EFs["Norma"] == "E5") | (EFs["Norma"] == "E6"))]
pme4e5 = EFs.loc[
    ((EFs.Modo.str.contains("CAM*")) | (EFs.Modo.str.contains("BUS*"))) & (EFs["Pollutant"] == "PM Exhaust [g/kWh]")]

pme4e5_2 = pme4e5.iloc[:,6:54]*consumo.iloc[:,6:54].values*0.278
namesPM = pme4e5.iloc[:,0:6]
pme4e5_3 = pd.concat([namesPM,pme4e5_2],axis= 1, ignore_index=False)
pme4e5_3["Pollutant"] = "PM Exhaust"

In [98]:
kilometraje = pd.read_excel("input/kilometraje.xlsx")
xls = pd.ExcelFile("input/N2O_VLCVLP.xlsx")
dfN2OVL = pd.read_excel(xls,"0-30", index_col=None)
#dfN2OVL["N2O"] = (dfN2OVL["a"]*kilometraje+ dfN2OVL["b"])*dfN2OVL["Base [mg/km]"]/1000
km = kilometraje.loc[((kilometraje["Modo"]=="VLP") | (kilometraje["Modo"]=="VLC"))]
mkm = km.merge(dfN2OVL, left_on=["Modo", "Motorizacion", "Norma"], right_on=["Modo","Motorizacion","Norma"], how="left")
N2O = mkm.iloc[:,7:54].multiply(mkm["a"],axis="index").add(mkm["b"],axis=0).multiply(mkm["Base [mg/km]"],axis="index")/1000
N2O = pd.concat([mkm.iloc[:,0:6],N2O],axis=1)
N2O["Pollutant"]="N2O"
xls2 = pd.ExcelFile("input/NH3_VLCVLP.xlsx")
dfNH3VL = pd.read_excel(xls2,"0-30", index_col=None)
mkm = km.merge(dfNH3VL, left_on=["Modo", "Motorizacion", "Norma"], right_on=["Modo","Motorizacion","Norma"], how="left")
NH3 = mkm.iloc[:,7:54].multiply(mkm["a"],axis="index").add(mkm["b"],axis=0).multiply(mkm["Base [mg/km]"],axis="index")/1000
NH3 = pd.concat([mkm.iloc[:,0:6],NH3],axis=1)
NH3["Pollutant"]="NH3"

In [115]:
EF = pd.concat([EFs,CO2,SOx,kmLt,pme4e5_3,N2O,NH3],axis=0) 

In [118]:
#Calculating BC
df_bc = pd.read_excel('input/factores_bc.xlsx')
df_bc.sort_values("Modo")
pm_concat = EF.loc[ EF["Pollutant"]=="PM Exhaust"]

df3 = pm_concat.merge(df_bc,left_on=["Modo","Motorizacion","Norma"],right_on=["Modo","Motorizacion","Norma"],how="left")
#df3["BC_f"] = df3["2014"] * df3["BC"]
#df2
BC = df3.iloc[:,6:53].multiply(df3["BC"], axis="index")
BC = pd.concat([df3.iloc[:,0:6], BC], axis=1)
BC["Pollutant"]="BC"

In [2]:
EF[EF["Modo"]=="BUS INT PROV"]["Norma"].unique()
EF["Pollutant"].unique()

NameError: name 'EF' is not defined

In [152]:
#Formateando a lo STEP 3 y agregando Taxi
EF = pd.concat([EFs,CO2,SOx,kmLt,pme4e5_3,N2O,NH3,BC],axis=0) 
EF.loc[EF["Modo"]=="VLP","Modo"] = "Vehículo Liviano Pasajero"
EF.loc[EF["Modo"]=="VLC","Modo"] = "Vehículo Liviano Comercial"
Taxi = EF.loc[(EF["Modo"]=="Vehículo Liviano Pasajero") & (EF["Ambito"]=="Urbano")].copy()
Taxi.loc[Taxi["Modo"]=="Vehículo Liviano Pasajero","Modo"] = "Taxi"
EF = pd.concat([EF,Taxi], axis= 0)
EF.loc[EF["Modo"]=="MOT","Modo"] = "Motocicleta"
EF.loc[EF["Modo"]=="BUS RIG","Modo"] = "Bus Rígido"
EF.loc[EF["Modo"]=="BUS INT PROV","Modo"] = "Bus Rígido"
EF.loc[EF["Modo"]=="BUS ART","Modo"] = "Bus Articulado"
EF.insert(6, "Unidad", "g/km")
EF.loc[EF["Region"]=="R13", "Region"] = "RM"
EF.loc[EF["Pollutant"]=="PM Exhaust", "Pollutant"] = "PM"
EF = EF.sort_values(["Pollutant","Region","Modo"])
EF.to_excel("EF.xlsx",index=False)